In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]

In [2]:
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=3)
search.invoke("tell me about 2025 LA wildfire updates")

[{'url': 'https://apnews.com/live/los-angeles-wildfires-updates-1-13-25',
  'content': 'Los Angeles wildfires: Updates from Jan. 13, 2025 | AP News 7 of 10\xa0|\xa0Los Angeles Fire Department’s Mike Alvarez works on extinguishing hot spots in the aftermath of the Palisades Fire as a Malibu resident, top right, watches the sunset from atop his beachfront home along the Pacific Coast Highway in Malibu, Calif., Sunday, Jan. 12, 2025. 7 of 10\xa0|\xa0Los Angeles Fire Department’s Mike Alvarez works on extinguishing hot spots in the aftermath of the Palisades Fire as a Malibu resident, top right, watches the sunset from atop his beachfront home along the Pacific Coast Highway in Malibu, Calif., Sunday, Jan. 12, 2025. “We’re absolutely better prepared for this coming,” Los Angeles County Fire Chief Anthony Marrone said when asked what will be different this time with similarly strong winds forecast as last week which fueled the massive fires.'},
 {'url': 'https://edition.cnn.com/weather/live

In [5]:
tools = [search]

In [6]:
tools

[TavilySearchResults(max_results=3)]

In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [8]:
from langchain_core.messages import HumanMessage
response = agent_executor.invoke({"messages": [HumanMessage(content="tell me about 2025 LA wildfire updates")]})
response

{'messages': [HumanMessage(content='tell me about 2025 LA wildfire updates', id='029265a1-dfda-492d-b1c4-13792efde72b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0pnUQuhJU7TiQ3H18HSZhRgO', 'function': {'arguments': '{"query":"2025 Los Angeles wildfire updates"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-51d00fc8-f63a-417c-8879-d1811f391e80-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 Los Angeles wildfire updates'}, 'id': 'call_0pnUQuhJU7TiQ3H18HSZhRgO', 'type': 'tool_call'}], usag

In [9]:
response["messages"]

[HumanMessage(content='tell me about 2025 LA wildfire updates', id='029265a1-dfda-492d-b1c4-13792efde72b'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0pnUQuhJU7TiQ3H18HSZhRgO', 'function': {'arguments': '{"query":"2025 Los Angeles wildfire updates"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-51d00fc8-f63a-417c-8879-d1811f391e80-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025 Los Angeles wildfire updates'}, 'id': 'call_0pnUQuhJU7TiQ3H18HSZhRgO', 'type': 'tool_call'}], usage_metadata={'i

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [10]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [11]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "001"}}

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E4dfQzVMzl8ZIw3AvgDqLZJv', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 91, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-90d57170-1481-45ce-bd9a-a63b0948a8c7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_E4dfQzVMzl8ZIw3AvgDqLZJv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 24, 'total_tokens': 115})]}}
----
{'tools': {'messages': [ToolMe

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='The top stars of the Spain national team, who won the 2024 Eurocup, included players like Mikel Oyarzabal, who scored the winning goal in the final against England. Other key players in the team might have included established stars like Sergio Ramos, Gerard Pique, Thiago Alcantara, and young talents like Pedri and Ferran Torres. The exact list of top stars can be found in detailed reports or analysis of the tournament.', response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 469, 'total_tokens': 564, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5a8f8757-5147-4ef8-bc43-21efeba8472c-0', usage_metadata={'input_tokens': 469, 'output_tokens': 95, 'total_

In [16]:
config = {"configurable": {"thread_id": "001"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='We were talking about the Spain national soccer team, which won the 2024 European Championship (Eurocup).', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 578, 'total_tokens': 602, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b216ed49-2146-4cfe-9ffc-8ef85ec40f15-0', usage_metadata={'input_tokens': 578, 'output_tokens': 24, 'total_tokens': 602})]}}
----
